<a href="https://colab.research.google.com/github/emmanuelalafaa97/Hamoye_Premiere_Project_Malaria/blob/main/HamoyePremiereProject_Malaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting High-Risk Zones for Malaria Outbreaks Malaria
##Hamoye Premiere Project, GAN group##





## 1. Introduction
### Project Instructions
In 2022, there were 249 million malaria cases globally, resulting in 608,000 deaths, with 76% affecting children under 5. Leveraging this data, develop predictive models to identify areas prone to malaria outbreaks, contributing to proactive measures in combating the disease.

Gen AI learners will analyze malaria data, develop predictive models, and evaluate their performance. By addressing real-world health challenges, learners gain practical experience in data analysis and machine learning while contributing to efforts to prevent malaria-related deaths.

### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Data Gathering


### 2.1 Child-Health-Coverage-Database-December-2023.xlsx file



#### 2.1.0 Description

Child-Health-Coverage-Database-December-2023.xlsx file contains comprehensive data for Child Health in long form, pivot form, global and regional aggregates and trends, as well as individual pivot tables per indicator. There are 14 indicators, 8 of them are related to MALARIA:
- IPTP - IPTP for pregnant women - Percentage of women (aged 15–49) who received three or more doses of intermittent preventive treatment during antenatal care visits during their last pregnancy
- ITN - ITN use by children - Percentage of children under age 5 who slept under an insecticide-treated mosquito net the night prior to the survey.
- ITN2 - Household with ITN and/or IRS - Percentage of households with at least one ITN for every two persons and /or IRS in the past 12 months
- ITNOWN - Percentage of households with at least one insecticide treated mosquito net (ITN)
- ITNPREG - Pregnant women sleeping under ITN - Percentage of pregnant women (age 15-49) who slept under an insecticide-treated net the previous night
- MLRCARE - Careseeking for febrile children - Percentage of children under age 5 with fever for whom advice or treatment was sought
- MLRDIAG - Malaria diagnostics - Percentage of febrile children under age 5 who had a finger or heel stick for malaria testing.
- MLRACT - Malaria, 1st line treatment- Percentage of febrile children under age 5 receiving ACT (first-line antimalarial drug), among those receiving any antimalarial drugs.

Another 6 indicators are related to Diarrhoea or Pneumonia:
- DIARCARE - Care seeking for diarrhoea - Percentage of children under age five who had diarrhoea in the two weeks preceding the survey for whom advice or treatment was sought from a health facility or provider
- ORS - Diarrhoea treatment with ORS -Percentage of children under age five who had diarrhoea in the two weeks preceding the survey and were given oral rehydration salts (ORS packets or pre-packaged ORS fluids)
- ORTCF - Diarrhoea treatment - Percentage of children under age 5 who had diarrhoea and received oral rehydration therapy (oral rehydration salts OR recommended homemade fluids OR increased fluids) AND continued feeding.
- ORSZINC - Diarrhoea treatment with ORS and Zinc - Percentage of children under age five who had diarrhoea in the two weeks preceding the survey and were given ORS and Zinc
- ZINC - Diarrhoea treatment with zinc - Percentage of children under age 5 with diarrhoea who received zinc supplements
- PNEUCARE - Care seeking for ARI - Percentage of children under age 5 with acute respiratory infection symptoms whom advice or treatment was sought from a health facility or provider.

For our Malaria project only the first **8 indicators are relevant**. We will use data in the excel sheets for the indicators for our project dataset.

**Which columns do we need?**

All indicator sheets have the same structure:
ISO, Countries and areas, UNICEF Reporting Region, UNICEF Programme Region, World Bank Income Group (2022), YEAR, Short Source, Long Source, National and stratifiers: Area(Rural/Urban), WIX(Poorest...Richest), Mother's Education(None, Primary, Sec & Higher). Columns "Countries and areas", "Year", "Income group" ,"National" are important for our project. We won't use stratifiers at this stage. Values in "National" column of each indicator's sheet will be taken to our final dataset, as well as country name, region, year, income group, which are common for all indicators.

#### 2.1.1 Read all sheets of Child-Health-Coverage-Database-December-2023.xlsx file

In [ ]:
dfs=pd.read_excel('/content/drive/MyDrive/Child-Health-Coverage-Database-December-2023.xlsx',sheet_name=None)

In [ ]:
# Read the Excel file
filepath = '/content/drive/MyDrive/Child-Health-Coverage-Database-December-2023.xlsx'
xls = pd.ExcelFile(filepath)

# Get the sheet names
sheet_names = xls.sheet_names

# Create a dictionary to store data frames for each sheet
dfs = {}

# Read each sheet into a data frame and store it in the dictionary
for sheet_name in sheet_names:
    dfs[sheet_name] = xls.parse(sheet_name)

In [ ]:
# Download each sheet as a separate Excel file
for sheet_name, df in dfs.items():
    df.to_excel(f'{sheet_name}.xlsx', index=False)

In [ ]:
sheet_names

['Index',
 'Long',
 'Pivot',
 'Global and Regional Aggregates',
 'Global and Regional Trends',
 'DIARCARE',
 'ORS',
 'ORTCF',
 'ORSZINC',
 'ZINC',
 'IPTP',
 'ITN',
 'ITN2',
 'ITNOWN',
 'ITNPREG',
 'MLRCARE',
 'MLRDIAG',
 'MLRACT',
 'PNEUCARE']

In [ ]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20'],
      dtype='object')

Access a specific data frame by sheet name

specific_df = dfs['Sheet1']  # Replace 'Sheet1' with the actual sheet name

print(specific_df.head())  # Display the first few rows of the specific data frame

#### 2.1.2. Write a function to process excel file : to take only relevant sheets, relevant columns, then merge them into one file

In [ ]:
def process_excel(filepath):
    xls_data = pd.ExcelFile(filepath)
    our_sheets = ['IPTP', 'ITN', 'ITN2', 'ITNOWN', 'ITNPREG', 'MLRCARE', 'MLRDIAG', 'MLRACT']
    our_cols = ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 8']

    dfs = []  # List to store DataFrames for selected sheets

    for sheet_name in xls_data.sheet_names:
        if sheet_name in our_sheets:
            parsed_sheet = xls_data.parse(sheet_name, header=5)  # Assuming header is at row index 5
            parsed_sheet = parsed_sheet[our_cols]  # Selecting only the desired columns
            parsed_sheet.columns = ['country', 'region', 'income_group', 'year', sheet_name]  # Renaming columns
            dfs.append(parsed_sheet)

    # Merging DataFrames using an outer join
    final_df = dfs[0]  # Start with the first DataFrame
    for df in dfs[1:]:
        final_df = pd.merge(final_df, df, on=['country', 'region', 'income_group', 'year'], how='outer')

    return final_df



In [ ]:

filepath = '/content/drive/MyDrive/Child-Health-Coverage-Database-December-2023.xlsx'
mlr = process_excel(filepath)
mlr.head()

,country,region,income_group,year,IPTP,ITN,ITN2,ITNOWN,ITNPREG,MLRCARE,MLRDIAG,MLRACT
0,Angola,Eastern and Southern Africa,Lower middle income,2016,19.0,21.7,12.5,30.9,23.0,50.8,34.3,76.7
1,Burundi,Eastern and Southern Africa,Low income,2010,0.1,45.3,22.1,52.0,49.7,62.1,27.0,69.5
2,Burundi,Eastern and Southern Africa,Low income,2017,12.6,39.9,NaN,46.1,43.9,69.6,66.4,NaN
3,Benin,West and Central Africa,Lower middle income,2012,8.8,69.7,45.7,79.8,74.5,42.9,17.3,32.0
4,Benin,West and Central Africa,Lower middle income,2014,12.5,72.7,40.1,77.0,47.0,43.7,19.1,13.1


In [ ]:
mlr.shape

(371, 12)

In [ ]:
mlr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       371 non-null    object 
 1   region        371 non-null    object 
 2   income_group  371 non-null    object 
 3   year          371 non-null    int64  
 4   IPTP          149 non-null    float64
 5   ITN           272 non-null    float64
 6   ITN2          118 non-null    float64
 7   ITNOWN        247 non-null    float64
 8   ITNPREG       188 non-null    float64
 9   MLRCARE       214 non-null    float64
 10  MLRDIAG       158 non-null    float64
 11  MLRACT        167 non-null    float64
dtypes: float64(8), int64(1), object(3)
memory usage: 34.9+ KB


#### 2.1.3 Adress missing values

In [ ]:
mlr.isnull().sum()

country           0
region            0
income_group      0
year              0
IPTP            222
ITN              99
ITN2            253
ITNOWN          124
ITNPREG         183
MLRCARE         157
MLRDIAG         213
MLRACT          204
dtype: int64

In [ ]:
# Check if there is any pattern in the distribution of NaNs by year.
# group by year and find number of missing values per each indicator per each year
mis=mlr.groupby('year').apply(lambda x: x.isnull().sum())
mis




,country,region,income_group,year,IPTP,ITN,ITN2,ITNOWN,ITNPREG,MLRCARE,MLRDIAG,MLRACT
year,,,,,,,,,,,,
2000,0,0,0,0,23,2,23,22,23,23,23,23
2001,0,0,0,0,2,0,2,2,2,2,2,2
2002,0,0,0,0,3,0,3,2,1,3,3,3
2003,0,0,0,0,2,0,4,0,0,4,4,4
2004,0,0,0,0,4,1,4,2,3,3,4,4
2005,0,0,0,0,7,1,12,0,5,12,12,9
2006,0,0,0,0,14,2,24,4,18,23,25,13
2007,0,0,0,0,14,5,17,5,10,11,17,11
2008,0,0,0,0,6,1,10,1,6,10,10,4


In [ ]:
# drop columns 'country',	'region',	'income_group',	'year' that have no missing values
mis=mis.drop(['country',	'region',	'income_group',	'year'],axis=1)
mis.head()

,IPTP,ITN,ITN2,ITNOWN,ITNPREG,MLRCARE,MLRDIAG,MLRACT
year,,,,,,,,
2000,23,2,23,22,23,23,23,23
2001,2,0,2,2,2,2,2,2
2002,3,0,3,2,1,3,3,3
2003,2,0,4,0,0,4,4,4
2004,4,1,4,2,3,3,4,4


In [ ]:
# Calculate the total count of rows per year, equal to the number of examined countries.
total_count = mlr.groupby('year').size().reset_index(name='total_count')
total_count

,year,total_count
0,2000,23
1,2001,2
2,2002,3
3,2003,4
4,2004,4
5,2005,12
6,2006,25
7,2007,17
8,2008,10
9,2009,17


In [ ]:
# add total_count column to mis table
mis_years=pd.merge(mis, total_count, on='year')
mis_years



,year,IPTP,ITN,ITN2,ITNOWN,ITNPREG,MLRCARE,MLRDIAG,MLRACT,total_count
0,2000,23,2,23,22,23,23,23,23,23
1,2001,2,0,2,2,2,2,2,2,2
2,2002,3,0,3,2,1,3,3,3,3
3,2003,2,0,4,0,0,4,4,4,4
4,2004,4,1,4,2,3,3,4,4,4
5,2005,7,1,12,0,5,12,12,9,12
6,2006,14,2,24,4,18,23,25,13,25
7,2007,14,5,17,5,10,11,17,11,17
8,2008,6,1,10,1,6,10,10,4,10
9,2009,12,5,16,5,11,12,9,8,17


We see that many indicators have all missing values for some years - in 2000-2005 number of rows equals to number of NaNs for 6 malaria indicators. Later on the share of NaNs per indicator goes down, because anti-malaria initiatives approved by WHO were spread more and more:

- IPTP: in 2004, the WHO recommended 2 doses of IPTp-SP during pregnancy, and in 2012 the WHO revised the policy on Intermittent Preventive Treatment with Sulphadoxine Pyrimethamine (IPTp-SP) to at least three doses to be taken at antenatal care (ANC) visits.
IPTP indicator in our table means "Percentage of women (aged 15–49) who received **three or more doses** of intermittent preventive treatment during antenatal care visits during their last pregnancy".  We may assume, that NaN values can be interpreted as 0 values for IPTP.

- MLRACT: In 2001, the WHO recommended artemisinin-based combination therapies (ACTs) as first-line therapy for uncomplicated malaria, but adopted widely since 2005.

- MLRDIAG:  UNICEF has actively been procuring mRDTs since 2008

- ITN, ITN2, ITNOWN, ITNPREG - widely spread after 2016 - 2018, when 197 million ITNs were delivered by manufacturers (https://www.who.int/news-room/feature-stories/detail/world-malaria-report-2019).

It becomes clear, that many countries in some years could not have non-zero values for certain indicators, while there are no '0' in df, but many NaNs.
So, actually missing values for malaria indicators reflect the absence of corresponding parameter, and **can be replaced with zero**.

##### A. Fill missing values with '0' and build mlr_big df

In [ ]:
# Fill missing values with 0 in mlr table and save in mlr_big df
mlr_big=mlr.fillna(0)
mlr_big.head()

,country,region,income_group,year,IPTP,ITN,ITN2,ITNOWN,ITNPREG,MLRCARE,MLRDIAG,MLRACT
0,Angola,Eastern and Southern Africa,Lower middle income,2016,19.0,21.7,12.5,30.9,23.0,50.8,34.3,76.7
1,Burundi,Eastern and Southern Africa,Low income,2010,0.1,45.3,22.1,52.0,49.7,62.1,27.0,69.5
2,Burundi,Eastern and Southern Africa,Low income,2017,12.6,39.9,0.0,46.1,43.9,69.6,66.4,0.0
3,Benin,West and Central Africa,Lower middle income,2012,8.8,69.7,45.7,79.8,74.5,42.9,17.3,32.0
4,Benin,West and Central Africa,Lower middle income,2014,12.5,72.7,40.1,77.0,47.0,43.7,19.1,13.1


In [ ]:
mlr_big.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       371 non-null    object 
 1   region        371 non-null    object 
 2   income_group  371 non-null    object 
 3   year          371 non-null    int64  
 4   IPTP          371 non-null    float64
 5   ITN           371 non-null    float64
 6   ITN2          371 non-null    float64
 7   ITNOWN        371 non-null    float64
 8   ITNPREG       371 non-null    float64
 9   MLRCARE       371 non-null    float64
 10  MLRDIAG       371 non-null    float64
 11  MLRACT        371 non-null    float64
dtypes: float64(8), int64(1), object(3)
memory usage: 34.9+ KB


In [ ]:
mlr_big['country'].nunique()

101

In [ ]:
mlr_big['country'].value_counts()

country
Togo          20
Senegal       14
Rwanda        10
Nigeria       10
Zambia         9
              ..
Costa Rica     1
Cabo Verde     1
Belize         1
Belarus        1
Lesotho        1
Name: count, Length: 101, dtype: int64

In [ ]:
# per region distribution
mlr_big['region'].value_counts()

region
West and Central Africa            152
Eastern and Southern Africa        105
East Asia and Pacific               40
Latin America and Caribbean         25
South Asia                          20
Eastern Europe and Central Asia     15
Sub-Saharan Africa                   7
Middle East and North Africa         6
Not Classified                       1
Name: count, dtype: int64

So, now our mlr_big df has 371 rows 12 columns, it containes data on 101 countries. Each country may appear in df 1-20 times (one survey per year), in years 2001 - 2022.

##### B. Drop missing values and build mlr_pure df

In [ ]:
#If we decide to drop Nans, we get_pure table
mlr_pure=mlr.dropna()
mlr_pure.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58 entries, 0 to 156
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       58 non-null     object 
 1   region        58 non-null     object 
 2   income_group  58 non-null     object 
 3   year          58 non-null     int64  
 4   IPTP          58 non-null     float64
 5   ITN           58 non-null     float64
 6   ITN2          58 non-null     float64
 7   ITNOWN        58 non-null     float64
 8   ITNPREG       58 non-null     float64
 9   MLRCARE       58 non-null     float64
 10  MLRDIAG       58 non-null     float64
 11  MLRACT        58 non-null     float64
dtypes: float64(8), int64(1), object(3)
memory usage: 5.9+ KB


In [ ]:
mlr_pure['country'].nunique()

32

In [ ]:
mlr_pure['country'].value_counts()

country
Senegal                             4
Malawi                              4
Ghana                               4
Nigeria                             3
Benin                               3
Burkina Faso                        3
Mozambique                          3
Kenya                               2
Mali                                2
Sierra Leone                        2
Liberia                             2
Gambia                              2
Uganda                              2
Cameroon                            2
Côte d'Ivoire                       2
Zambia                              2
Chad                                1
Papua New Guinea                    1
Sao Tome and Principe               1
Togo                                1
United Republic of Tanzania         1
Angola                              1
Madagascar                          1
Namibia                             1
Burundi                             1
Lao People's Democratic Republic    1
Guin

In [ ]:
mlr_pure['region'].value_counts()

region
West and Central Africa        36
Eastern and Southern Africa    20
East Asia and Pacific           2
Name: count, dtype: int64

In [ ]:
# find countries from 'East Asia and Pacific'  region
mlr_pure[mlr_pure['region']=='East Asia and Pacific']['country'].unique()

array(["Lao People's Democratic Republic", 'Papua New Guinea'],
      dtype=object)

In [ ]:
mlr_pure['year'].value_counts()

year
2018    13
2014     8
2016     7
2017     7
2019     5
2013     5
2010     2
2012     2
2021     2
2011     2
2020     2
2015     2
2009     1
Name: count, dtype: int64

So, after dropping rows with missing values we got mlr_pure  df, which has 58 rows 12 columns.  It contains data on 32 countries, for which we know all indicators related to malaria for each survey. Countries appear in df 1-4 times, in years 2009 - 2018, and belong to 3 regions: West and Central Africa (36),
Eastern and Southern Africa(20), East Asia and Pacific (2).

### 2.2. Read UNICEF-CME_CAUSE_OF_DEATH.csv

 This file contains data gathered by UN Inter-agency Group for Child Mortality Estimation. We filter it for malaria and get child mortality estimates for children under 5 years for all countries.

In [ ]:
death=pd.read_excel('/content/UNICEF-CME_CAUSE_OF_DEATH-All.xlsx')
death.head()

,DATAFLOW,REF_AREA:Geographic area,INDICATOR:Indicator,CAUSE_OF_DEATH:Cause of death,SEX:Sex,AGE_GROUP:Age group,SERIES_NAME:Series Name,TIME_PERIOD:Time period,OBS_VALUE:Observation value,UNIT_MEASURE:Unit of measure,MODEL:Model Used,REF_DATE:Reference Date,LOWER_BOUND:Lower Bound,UPPER_BOUND:Upper Bound
0,UNICEF:CME_CAUSE_OF_DEATH(1.0): CME Cause of d...,AFG: Afghanistan,DEATHS: Deaths,MALARIA: Malaria,_T: Total,Y0T4: Under 5 years,JHU,2000,266.0,D: Number of deaths,NaN,2000,148.0,463.0
1,UNICEF:CME_CAUSE_OF_DEATH(1.0): CME Cause of d...,AFG: Afghanistan,DEATHS: Deaths,MALARIA: Malaria,_T: Total,Y0T4: Under 5 years,JHU,2001,265.0,D: Number of deaths,NaN,2001,146.0,463.0
2,UNICEF:CME_CAUSE_OF_DEATH(1.0): CME Cause of d...,AFG: Afghanistan,DEATHS: Deaths,MALARIA: Malaria,_T: Total,Y0T4: Under 5 years,JHU,2002,317.0,D: Number of deaths,NaN,2002,187.0,516.0
3,UNICEF:CME_CAUSE_OF_DEATH(1.0): CME Cause of d...,AFG: Afghanistan,DEATHS: Deaths,MALARIA: Malaria,_T: Total,Y0T4: Under 5 years,JHU,2003,208.0,D: Number of deaths,NaN,2003,121.0,340.0
4,UNICEF:CME_CAUSE_OF_DEATH(1.0): CME Cause of d...,AFG: Afghanistan,DEATHS: Deaths,MALARIA: Malaria,_T: Total,Y0T4: Under 5 years,JHU,2004,83.0,D: Number of deaths,NaN,2004,45.0,141.0


In [ ]:
death.columns

Index(['DATAFLOW', 'REF_AREA:Geographic area', 'INDICATOR:Indicator',
       'CAUSE_OF_DEATH:Cause of death', 'SEX:Sex', 'AGE_GROUP:Age group',
       'SERIES_NAME:Series Name', 'TIME_PERIOD:Time period',
       'OBS_VALUE:Observation value', 'UNIT_MEASURE:Unit of measure',
       'MODEL:Model Used', 'REF_DATE:Reference Date',
       'LOWER_BOUND:Lower Bound', 'UPPER_BOUND:Upper Bound'],
      dtype='object')

In [ ]:
# select relevant columns: REF_AREA:Geographic area, TIME_PERIOD:Time period	OBS_VALUE:Observation, UNIT_MEASURE:Unit of measure'  and rename them
dth=death[['REF_AREA:Geographic area','TIME_PERIOD:Time period','OBS_VALUE:Observation value', 'UNIT_MEASURE:Unit of measure']]
dth.columns=['country','year','died','unit']
dth.sample(10)

,country,year,died,unit
6310,KWT: Kuwait,2018,0.000000,PCNT: %
3283,DOM: Dominican Republic,2005,0.014450,"D_PER_1000_B: Deaths per 1,000 live births"
9642,ROU: Romania,2005,0.000000,PCNT: %
11127,TGO: Togo,2010,3076.331425,D: Number of deaths
832,BEL: Belgium,2018,0.000000,PCNT: %
7318,MDV: Maldives,2014,0.000000,"D_PER_1000_B: Deaths per 1,000 live births"
5181,IND: India,2011,0.004280,PCNT: %
9368,PRK: Democratic People's Republic of Korea,2017,0.000000,D: Number of deaths
9686,RUS: Russian Federation,2005,0.000000,D: Number of deaths
2314,COD: Democratic Republic of the Congo,2004,108245.673300,D: Number of deaths


In [ ]:
# delete abbreviation from country names
dth['country'] = dth['country'].str.split(': ').str[1]
dth.head()

<ipython-input-35-45b906f2da96>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dth['country'] = dth['country'].str.split(': ').str[1]


,country,year,died,unit
0,Afghanistan,2000,266.0,D: Number of deaths
1,Afghanistan,2001,265.0,D: Number of deaths
2,Afghanistan,2002,317.0,D: Number of deaths
3,Afghanistan,2003,208.0,D: Number of deaths
4,Afghanistan,2004,83.0,D: Number of deaths


In [ ]:
# We need death estimate as percentage. Extract rows where the value in unit column  equals 'PCNT: %':
dth_pcnt=dth[dth['unit']=='PCNT: %']
dth_pcnt.head()

,country,year,died,unit
22,Afghanistan,2000,0.00171,PCNT: %
23,Afghanistan,2001,0.00192,PCNT: %
24,Afghanistan,2002,0.00248,PCNT: %
25,Afghanistan,2003,0.00200,PCNT: %
26,Afghanistan,2004,0.00080,PCNT: %


In [ ]:
# drop unit column:
dth_pcnt=dth_pcnt.drop('unit',axis=1)
dth_pcnt.head()


,country,year,died
22,Afghanistan,2000,0.00171
23,Afghanistan,2001,0.00192
24,Afghanistan,2002,0.00248
25,Afghanistan,2003,0.00200
26,Afghanistan,2004,0.00080


In [ ]:
#show values in died column as percents(multiply by 100 and round to 1 decimal place)
dth_pcnt['died']=dth_pcnt['died'].apply(lambda x: round(x*100,1))
dth_pcnt.head(2)

,country,year,died
22,Afghanistan,2000,0.2
23,Afghanistan,2001,0.2


In [ ]:
dth_pcnt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4488 entries, 22 to 13404
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  4488 non-null   object 
 1   year     4488 non-null   int64  
 2   died     4488 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 140.2+ KB


### 2.3 Merge data in mlr table for malaria indicators and dth_pcnt table for death percentage by country and year

#### A. From mlr_big to our_big

In [ ]:
# Merge data in df table for malaria indicators and dth_pcnt table for death percentage by country and year
our_big = pd.merge(mlr_big, dth_pcnt, on=['country', 'year'])
our_big.head(2)


,country,region,income_group,year,IPTP,ITN,ITN2,ITNOWN,ITNPREG,MLRCARE,MLRDIAG,MLRACT,died
0,Angola,Eastern and Southern Africa,Lower middle income,2016,19.0,21.7,12.5,30.9,23.0,50.8,34.3,76.7,7.4
1,Burundi,Eastern and Southern Africa,Low income,2010,0.1,45.3,22.1,52.0,49.7,62.1,27.0,69.5,6.8


In [ ]:
our_big.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       361 non-null    object 
 1   region        361 non-null    object 
 2   income_group  361 non-null    object 
 3   year          361 non-null    int64  
 4   IPTP          361 non-null    float64
 5   ITN           361 non-null    float64
 6   ITN2          361 non-null    float64
 7   ITNOWN        361 non-null    float64
 8   ITNPREG       361 non-null    float64
 9   MLRCARE       361 non-null    float64
 10  MLRDIAG       361 non-null    float64
 11  MLRACT        361 non-null    float64
 12  died          361 non-null    float64
dtypes: float64(9), int64(1), object(3)
memory usage: 36.8+ KB


In [ ]:
our_big.region.value_counts()

region
West and Central Africa            145
Eastern and Southern Africa        105
East Asia and Pacific               39
Latin America and Caribbean         24
South Asia                          20
Eastern Europe and Central Asia     15
Sub-Saharan Africa                   7
Middle East and North Africa         6
Name: count, dtype: int64

In [ ]:
our_big.country.value_counts()

country
Togo         20
Senegal      14
Rwanda       10
Nigeria      10
Zambia        9
             ..
Lesotho       1
Egypt         1
Iraq          1
Fiji          1
Argentina     1
Name: count, Length: 98, dtype: int64

#### B. From mlr_pure to our_pure

In [ ]:
# Merge data in df table for malaria indicators and dth_pcnt table for death percentage by country and year
our_pure = pd.merge(mlr_pure, dth_pcnt, on=['country', 'year'])
our_pure.head(2)


,country,region,income_group,year,IPTP,ITN,ITN2,ITNOWN,ITNPREG,MLRCARE,MLRDIAG,MLRACT,died
0,Angola,Eastern and Southern Africa,Lower middle income,2016,19.0,21.7,12.5,30.9,23.0,50.8,34.3,76.7,7.4
1,Burundi,Eastern and Southern Africa,Low income,2010,0.1,45.3,22.1,52.0,49.7,62.1,27.0,69.5,6.8


In [ ]:
our_pure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       56 non-null     object 
 1   region        56 non-null     object 
 2   income_group  56 non-null     object 
 3   year          56 non-null     int64  
 4   IPTP          56 non-null     float64
 5   ITN           56 non-null     float64
 6   ITN2          56 non-null     float64
 7   ITNOWN        56 non-null     float64
 8   ITNPREG       56 non-null     float64
 9   MLRCARE       56 non-null     float64
 10  MLRDIAG       56 non-null     float64
 11  MLRACT        56 non-null     float64
 12  died          56 non-null     float64
dtypes: float64(9), int64(1), object(3)
memory usage: 5.8+ KB


In [ ]:
our_pure.country.nunique()

31

In [ ]:
our_pure.region.value_counts()

region
West and Central Africa        34
Eastern and Southern Africa    20
East Asia and Pacific           2
Name: count, dtype: int64

#### Compare our_big and our_pure

In [ ]:
our_big.describe()

,year,IPTP,ITN,ITN2,ITNOWN,ITNPREG,MLRCARE,MLRDIAG,MLRACT,died
count,361.000000,361.000000,361.000000,361.000000,361.000000,361.000000,361.000000,361.000000,361.000000,361.000000
mean,2012.138504,7.650693,26.556233,11.155956,34.270914,20.754848,34.958726,10.473961,15.891413,8.769252
std,5.716709,13.698015,25.934522,18.710966,32.240530,26.083598,31.891972,15.250965,26.104887,9.495614
min,2000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2008.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000
50%,2013.000000,0.000000,20.000000,0.000000,32.000000,1.400000,44.600000,0.000000,0.000000,4.100000
75%,2017.000000,10.300000,49.800000,23.100000,65.800000,46.100000,63.200000,17.600000,21.400000,14.700000
max,2021.000000,67.300000,95.500000,94.800000,96.600000,94.500000,92.900000,66.400000,98.700000,45.600000


In [ ]:
our_pure.describe()

,year,IPTP,ITN,ITN2,ITNOWN,ITNPREG,MLRCARE,MLRDIAG,MLRACT,died
count,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000
mean,2015.839286,24.464286,53.132143,39.367857,68.216071,52.303571,59.053571,29.942857,48.607143,12.982143
std,2.833416,15.642035,15.657436,13.726287,14.305686,16.185167,13.546727,15.252396,30.345063,9.115444
min,2009.000000,0.100000,5.600000,12.500000,24.400000,3.600000,31.500000,4.900000,1.500000,0.000000
25%,2014.000000,13.375000,46.275000,28.700000,60.075000,44.275000,49.500000,16.325000,20.750000,6.350000
50%,2016.500000,21.800000,54.100000,39.900000,69.750000,52.400000,59.700000,27.550000,42.400000,10.850000
75%,2018.000000,32.675000,62.075000,45.750000,78.325000,61.025000,70.550000,41.600000,78.950000,19.750000
max,2021.000000,61.000000,93.600000,94.800000,96.600000,94.500000,87.000000,65.000000,98.600000,45.600000


With these our_pure and our_big datasets we can build  models for predicting regions with malaria outbreaks.

The datasets differ not only in number of countries and years encompassed, but also in the distribution of parameters:
- range of values is bigger in our_big: the most evident distinction is that min values for malaria indicators in our_big are '0', while there is no zeros in our_pure df
- max values often are the same, but sometimes differ in both directions, but not significantly
- mean, 25%, 50%, 75% are always greater in our_pure than in our_big for all malaria indicators and 'died' column.

This difference in the distribution is explained by the fact that our_pure includes mostly African countries, where the situation with malaria is more severe, while in our_big df there are countries from 8 regions, and in many of them malaria is not widely spread, so average values are lower. But it is interesting to build a model that takes 'region' as a feature and to predict malaria outbreaks in any country, any region.

In [ ]:
# Convert df to a CSV file
our_big.to_csv('our_big.csv', index=False)

In [ ]:
our_pure.to_csv('our_pure.csv', index=False)

###  Read '2000' and '2021' files with data on percentage of deaths caused by malaria in children under 5 years of age in Africa

In [ ]:
df_00 = pd.read_csv('/content/drive/MyDrive/2000.csv',header=None)
df_00.head()

,0,1,2,3,4,5
0,Algeria,0,Middle East and North Africa,27.5231 3.1685,Algeria,2000
1,Angola,14,Sub-Saharan Africa,-12.2916 17.5029,Angola,2000
2,Benin,18,Sub-Saharan Africa,10.6795 2.2845,Benin,2000
3,Botswana,0,Sub-Saharan Africa,-21.6721 23.9051,Botswana,2000
4,Burkina Faso,41,Sub-Saharan Africa,13.0516 -0.6541,Burkina Faso,2000


In [ ]:
df_21 = pd.read_csv('/content/drive/MyDrive/2021.csv', header=None)
df_21.head()

,0,1,2,3,4,5
0,Algeria,0,Middle East and North Africa,27.5231 3.1685,Algeria,2021
1,Angola,10,Sub-Saharan Africa,-12.2916 17.5029,Angola,2021
2,Benin,25,Sub-Saharan Africa,10.6795 2.2845,Benin,2021
3,Botswana,0,Sub-Saharan Africa,-21.6721 23.9051,Botswana,2021
4,Burkina Faso,24,Sub-Saharan Africa,13.0516 -0.6541,Burkina Faso,2021


We can extract data on deaths peecentage for years 2000 and 2021 for each country by merging tables on country names column  and taking only necessary columns.

In [ ]:
merged_table = pd.merge(df_00, df_21, on=4)
merged_table.head()

,0_x,1_x,2_x,3_x,4,5_x,0_y,1_y,2_y,3_y,5_y
0,Algeria,0,Middle East and North Africa,27.5231 3.1685,Algeria,2000,Algeria,0,Middle East and North Africa,27.5231 3.1685,2021
1,Angola,14,Sub-Saharan Africa,-12.2916 17.5029,Angola,2000,Angola,10,Sub-Saharan Africa,-12.2916 17.5029,2021
2,Benin,18,Sub-Saharan Africa,10.6795 2.2845,Benin,2000,Benin,25,Sub-Saharan Africa,10.6795 2.2845,2021
3,Botswana,0,Sub-Saharan Africa,-21.6721 23.9051,Botswana,2000,Botswana,0,Sub-Saharan Africa,-21.6721 23.9051,2021
4,Burkina Faso,41,Sub-Saharan Africa,13.0516 -0.6541,Burkina Faso,2000,Burkina Faso,24,Sub-Saharan Africa,13.0516 -0.6541,2021


In [ ]:
died = merged_table[[4, '1_x', '1_y']]
died.columns = ['country', 'died_2000', 'died_2021']
died.head()

,country,died_2000,died_2021
0,Algeria,0,0
1,Angola,14,10
2,Benin,18,25
3,Botswana,0,0
4,Burkina Faso,41,24
